In [ ]:
from datetime import datetime
import time 
import ee


def ee_monitoring(ee_task):
  while ee_task.active():
    print('Task polling (id: {}).'.format(ee_task.id))
    time.sleep(5)    
    
def rusle(roi, prefix, folder, scale):
    """ Estimates Erosion using RUSLE
    Args:
        roi (ee.Geometry): Study area
        prefix: Word to be added at the beginning of each downloaded image
        folder: Folder in Google Drive where the results will be exported
        scale: Working scale of the project
    """
    #-----------
    # R factor
    #-----------

    clim_rainmap = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01").clip(roi)
    year = clim_rainmap.reduce(ee.Reducer.sum())
    R_monthly = ee.Image(10).pow(ee.Image(1.5).multiply(clim_rainmap.pow(2).divide(year).log10().subtract(-0.08188))).multiply(1.735)
    factorR = R_monthly.reduce(ee.Reducer.sum()).rename('factorR')
    
    #-----------
    # K factor
    #-----------

    # Load all necessary information to estimate the K factor
    sand = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
    silt = ee.Image('users/aschwantes/SLTPPT_I').divide(100).clip(roi)
    clay = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0').clip(roi)
    morg = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').multiply(0.58).clip(roi)
    sn1 = sand.expression('1 - b0 / 100', {'b0': sand})
    orgcar = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').clip(roi)
    # Combining all images into one
    soil = ee.Image([sand, silt, clay, morg, sn1, orgcar]).rename(['sand', 'silt', 'clay', 'morg', 'sn1', 'orgcar'] )
    factorK = soil.expression(
        '(0.2 + 0.3 * exp(-0.0256 * SAND * (1 - (SILT / 100)))) * (1 - (0.25 * CLAY / (CLAY + exp(3.72 - 2.95 * CLAY)))) * (1 - (0.7 * SN1 / (SN1 + exp(-5.51 + 22.9 * SN1))))',
        {
        'SAND': soil.select('sand'),
        'SILT': soil.select('silt'),
        'CLAY': soil.select('clay'),
        'MORG': soil.select('morg'),
        'SN1':  soil.select('sn1'),
        'CORG': soil.select('orgcar')
        }).rename('factorK')
    
    #-----------
    # LS factor
    #-----------

    facc = ee.Image("WWF/HydroSHEDS/15ACC").clip(roi)
    dem = ee.Image("WWF/HydroSHEDS/03CONDEM").clip(roi)
    slope = ee.Terrain.slope(dem)

    ls_factors = ee.Image([facc, slope]).rename(['facc','slope'])

    factorLS = ls_factors.expression(
        '(FACC*270/22.13)**0.4*(SLOPE/0.0896)**1.3',
        {
        'FACC': ls_factors.select('facc'),
        'SLOPE': ls_factors.select('slope')     
        }).rename('factorLS')
    
    #-----------
    # C factor
    #-----------
    ndvi_median = ee.ImageCollection("MODIS/006/MOD13A2").filterBounds(roi).median().multiply(0.0001).select('NDVI').clip(roi)
    factorC = ee.Image(0.805).multiply(ndvi_median).multiply(-1).add(0.431).rename('factorC')

    erosion = factorC.multiply(factorR).multiply(factorLS).multiply(factorK).rename('A')
    erosion_output = ee.Image([erosion,factorC,factorR,factorLS,factorK])

    #-----------
    # Data download
    #-----------
    task = ee.batch.Export.image.toDrive(
        image=erosion_output,
        description= prefix+'output',
        folder=folder,     
        scale=scale,
        region = roi)  
    task.start()
    print('Downloading erosion data (A)')
    return task
